# Tensorflow_Mnist_BPN

In [2]:
import tensorflow
import tensorflow as tf
import numpy as np

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
train_data = np.float16(mnist_train[0])
train_labels = np.asarray(mnist_train[1], dtype=np.int32)
eval_data = np.float16(mnist_test[0])
eval_labels = np.asarray(mnist_test[1], dtype=np.int32)

a_0 = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

middle = 30
w_1 = tf.Variable(tf.truncated_normal([784, middle]))
b_1 = tf.Variable(tf.truncated_normal([1, middle]))
w_2 = tf.Variable(tf.truncated_normal([middle, 10]))
b_2 = tf.Variable(tf.truncated_normal([1, 10]))

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def sigma(x):
    return tf.div(tf.constant(1.0), tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))

# Forward propagation

In [4]:
z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)

In [5]:
diff = tf.subtract(a_2, y)

In [6]:
def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))

# Backward propagation

In [7]:
d_z_2 = tf.multiply(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)

In [8]:
eta = tf.constant(0.5)
step = [tf.assign(w_1,tf.subtract(w_1, tf.multiply(eta, d_w_1)))
  , tf.assign(b_1,tf.subtract(b_1, tf.multiply(eta,tf.reduce_mean(d_b_1, axis=[0]))))
  , tf.assign(w_2,tf.subtract(w_2, tf.multiply(eta, d_w_2)))
  , tf.assign(b_2,tf.subtract(b_2, tf.multiply(eta,tf.reduce_mean(d_b_2, axis=[0]))))
]

In [ ]:
acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    batch_xs, batch_ys = mnist_train.next_batch(10)
    sess.run(step, feed_dict = {a_0: batch_xs,y : batch_ys})
    if i % 1000 == 0:
        res = sess.run(acct_res, feed_dict = {a_0: mnist_test.images[:1000],y : mnist_test.labels[:1000]})
        print(res)